In [ ]:
# ==============================================================================
# 1. IMPORTAÇÕES E LOGGER
# ==============================================================================

import os
import time
import logging
from typing import List
import pandas as pd
import csv
import unicodedata
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

def setup_logging(log_file: str = 'logs/pacatuba/extracao_pacatuba.log') -> logging.Logger:
    # Cria a estrutura de diretórios se não existir
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    
    logger = logging.getLogger('osr_pacatuba')
    logger.setLevel(logging.INFO) # Mude para DEBUG para ver mais detalhes

    # Limpa handlers existentes para evitar duplicação de logs
    if logger.hasHandlers():
        logger.handlers.clear()

    formatter = logging.Formatter("[%(asctime)s] - %(levelname)s - [%(funcName)s] - %(message)s")

    # Handler para o arquivo de log
    file_handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
    file_handler.setFormatter(formatter)

    # Handler para o console
    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

logger = setup_logging()

In [ ]:
# ==============================================================================
# 2. CONSTANTES E FUNÇÕES AUXILIARES
# ==============================================================================

TERMOS_ROYALTIES = ["royalty", "royaltie", "petroleo"]

RE_REMOVE_PUNCTUATION = re.compile(r'[^a-zA-Z0-9\s]')

def normalizar(texto: str) -> str:
    """
    Normaliza um texto removendo acentos, pontuações e convertendo para minúsculas.
    """
    if not isinstance(texto, str):
        return ""
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = RE_REMOVE_PUNCTUATION.sub('', texto)
    return texto.lower()

In [ ]:
# ==============================================================================
# 3. DRIVER E INTERAÇÕES COM A PÁGINA
# ==============================================================================

def start_driver(headless=False) -> webdriver.Chrome:
    logger.info("Iniciando driver do Chrome...")
    options = webdriver.ChromeOptions()

    if headless:
        options.add_argument("--headless=new")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
    else:
        options.add_argument('--window-size=1920,1080')
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.get("https://transparencia.pacatuba.se.gov.br/public/portal/despesas")
    return driver

def selecionar_dropdown(driver, container_id, texto):
    try:
        logger.info(f"Selecionando: {texto}")
        trigger = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[@aria-labelledby='{container_id}']"))
        )
        trigger.click()
        opcao = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//li[contains(@class, 'select2-results__option') and normalize-space(.)='{texto}']"))
        )
        opcao.click()
    except Exception as e:
        logger.error(f"Erro ao selecionar {texto} no campo {container_id}: {e}")
        raise

def ir_para_proxima_pagina(driver):
    """
    Tenta clicar no botão 'Próxima Página'. Retorna True se conseguiu, False caso contrário.
    """
    try:
        # O botão é um <a> com uma tag <i> de classe 'next' dentro
        proxima_pagina_locator = (By.XPATH, "//a[contains(@class, 'page-link')][i[contains(@class, 'next')]]")
        
        botao = driver.find_element(*proxima_pagina_locator)
        # Verifica se o botão está desabilitado pelo seu elemento pai <li>
        parent_li = botao.find_element(By.XPATH, "./parent::li")
        
        if "disabled" in parent_li.get_attribute("class"):
            logger.info("Última página alcançada.")
            return False

        botao.click()
        logger.info("Navegando para a próxima página de resultados.")
        # Espera a tabela de resultados da nova página carregar
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table/tbody")))
        return True
    except NoSuchElementException:
        logger.info("Botão 'Próxima Página' não encontrado. Fim da paginação.")
        return False

In [ ]:
# ==============================================================================
# 4. EXTRAÇÃO LINHA A LINHA E PROCESSAMENTO DE PÁGINA
# ==============================================================================

def processar_pagina_de_resultados(driver, dados_coletados_ano):
    """
    Processa todas as linhas da página de resultados atual, extraindo dados
    de pagamentos de royalties.

    Args:
        driver (webdriver.Chrome): A instância do driver.
        dados_coletados_ano (list): A lista onde os dicionários com dados
                                     serão armazenados.
    """
    logger.info("Processando nova página de resultados...")
    
    # Encontra o número de linhas na página atual para iterar por índice
    try:
        num_linhas = len(WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//table/tbody/tr"))
        ))
        logger.info(f"Encontradas {num_linhas} linhas nesta página.")
    except TimeoutException:
        logger.warning("Nenhuma linha encontrada na tabela desta página.")
        return

    for i in range(num_linhas):
        try:
            # Re-localiza a linha a cada iteração para evitar StaleElementReferenceException
            linha_atual = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f"//table/tbody/tr[{i+1}]"))
            )

            # Extrai dados visíveis na tabela principal antes de clicar
            celulas = linha_atual.find_elements(By.TAG_NAME, "td")
            dados_preliminares = {
                'credor': celulas[1].text, # Exemplo, ajuste o índice da coluna
                'valor_pago': celulas[3].text # Exemplo, ajuste o índice da coluna
            }
            logger.debug(f"Processando linha {i+1}: Credor - {dados_preliminares['credor']}")

            # Localiza e clica no botão de detalhes
            botao_detalhes = linha_atual.find_element(By.CSS_SELECTOR, "a.btn-detalhes")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", botao_detalhes)
            time.sleep(0.3)
            botao_detalhes.click()
            
            # Aguarda a nova página de detalhes carregar
            # **ATENÇÃO**: O seletor abaixo é um EXEMPLO. Precisamos de um seletor real da página de detalhes.
            WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, "div.card-body")) 
            )

            # --- DENTRO DA PÁGINA DE DETALHES ---
            
            # **ATENÇÃO**: Os seletores abaixo são placeholders e precisam ser preenchidos
            # com os seletores reais da página de detalhes.
            try:
                fonte_recurso_element = driver.find_element(By.XPATH, "...") # Ex: //div[contains(., 'Fonte de Recurso')]/following-sibling::div
                fonte_recurso_texto = fonte_recurso_element.text
                
                if any(termo in normalizar(fonte_recurso_texto) for termo in TERMOS_ROYALTIES):
                    logger.info("Encontrado pagamento de Royalties. Extraindo dados completos...")
                    
                    dados_completos = dados_preliminares.copy() # Começa com os dados da tabela
                    dados_completos['fonte_recurso'] = fonte_recurso_texto
                    # Extraia outros campos da página de detalhes
                    # dados_completos['empenho'] = driver.find_element(By.XPATH, "...").text
                    # dados_completos['data_pagamento'] = driver.find_element(By.XPATH, "...").text
                    
                    dados_coletados_ano.append(dados_completos)
            except Exception as e_detalhes:
                logger.error(f"Erro ao extrair dados da página de detalhes: {e_detalhes}")

            # Volta para a página de resultados
            driver.back()

            # Aguarda a tabela de resultados recarregar após voltar
            WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.XPATH, "//table/tbody"))
            )

        except Exception as e_linha:
            logger.error(f"Erro ao processar a linha {i+1} da página. Pulando para a próxima. Erro: {e_linha}")
            continue

In [ ]:
# ==============================================================================
# 5. ORQUESTRAÇÃO E EXECUÇÃO
# ==============================================================================

def processar_ano(ano: str):
    logger.info(f"================ INICIANDO PROCESSAMENTO PARA O ANO DE {ano} ================")
    driver = None
    dados_coletados_do_ano = []
    
    try:
        driver = start_driver(headless=False) # Mude para True para rodar em segundo plano
        WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))

        # Aplica os filtros iniciais
        selecionar_dropdown(driver, "select2-tipo-container", "Pagamento")
        selecionar_dropdown(driver, "select2-ano-container", ano)

        # Clica no botão "Buscar" principal
        # **ATENÇÃO**: Confirme se este seletor ainda é o correto
        logger.info("Clicando no botão 'Buscar' para carregar os resultados iniciais.")
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button#filtrar.btn-buscar"))
        ).click()

        # Espera a primeira página de resultados carregar
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table/tbody")))
        logger.info("Tabela de resultados carregada.")

        # Loop de paginação
        pagina_atual = 1
        while True:
            logger.info(f"Processando página de resultados número: {pagina_atual}")
            processar_pagina_de_resultados(driver, dados_coletados_do_ano)
            
            if not ir_para_proxima_pagina(driver):
                break # Sai do loop se não houver mais páginas
            pagina_atual += 1

        # Salva os dados coletados em um arquivo CSV
        if dados_coletados_do_ano:
            output_dir = os.path.join("dados_finais", "pacatuba")
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, f"royalties_pacatuba_{ano}.csv")
            
            df = pd.DataFrame(dados_coletados_do_ano)
            df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
            logger.info(f"Processamento concluído. {len(df)} registros salvos em: {output_path}")
        else:
            logger.info("Nenhum registro de royalties encontrado para o ano.")

    except Exception as e:
        logger.critical(f"Erro CRÍTICO ao processar o ano {ano}: {e}")
    finally:
        if driver:
            driver.quit()
        logger.info(f"================ FINALIZADO PROCESSAMENTO DE {ano} ================")

# Bloco de execução principal
if __name__ == "__main__":
    ANOS_PARA_PROCESSAR = ["2018"] # Defina aqui os anos que deseja processar

    for ano in ANOS_PARA_PROCESSAR:
        processar_ano(ano)

In [3]:
# ==============================================================================
# 1. IMPORTAÇÕES E LOGGER
# ==============================================================================

import os
import time
import logging
from typing import List
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

def setup_logging(log_file: str = '../logs/pacatuba/extracao_pacatuba.log') -> logging.Logger:
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logger = logging.getLogger('osr_pacatuba')
    logger.setLevel(logging.INFO)

    if logger.hasHandlers():
        logger.handlers.clear()

    formatter = logging.Formatter("[%(asctime)s] - %(levelname)s - [%(funcName)s] - %(message)s")

    file_handler = logging.FileHandler(log_file, mode='a', encoding='utf-8')
    file_handler.setFormatter(formatter)

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger

logger = setup_logging()

# ==============================================================================
# 2. CONSTANTES
# ==============================================================================

TERMOS_ROYALTIES = ["1340040000", "1340050000", "1340060000", "1340070000","1340080000", "1721224000", "1721227000"]

# ==============================================================================
# 3. DRIVER E DROPDOWNS
# ==============================================================================

def start_driver(headless=False, download_dir: str = None) -> webdriver.Chrome:
    if not download_dir:
        raise ValueError("Diretório de download deve ser informado.")
    
    logger.info("Iniciando driver do Chrome...")
    options = webdriver.ChromeOptions()
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)

    if headless:
        options.add_argument("--headless=new")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
    else:
        options.add_argument('--window-size=1920,1080')
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.get("https://transparencia.pacatuba.se.gov.br/public/portal/despesas")
    return driver

def selecionar_dropdown(driver, container_id, texto):
    try:
        logger.info(f"Selecionando: {texto}")
        trigger = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[@aria-labelledby='{container_id}']"))
        )
        trigger.click()
        opcao = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//li[contains(@class, 'select2-results__option') and normalize-space(.)='{texto}']"))
        )
        opcao.click()
    except Exception as e:
        logger.error(f"Erro ao selecionar {texto} no campo {container_id}: {e}")
        raise

# ==============================================================================
# 4. CLASSIFICAÇÕES ECONÔMICAS (AJAX) COM LOOP
# ==============================================================================

def selecionar_classificacoes_economicas(driver, termos: List[str]):
    """
    Seleciona uma lista de classificações econômicas (fontes de recurso)
    buscando termo por termo e clicando na primeira opção resultante.

    Args:
        driver (webdriver.Chrome): A instância do driver do Selenium.
        termos (List[str]): Lista de termos exatos a serem pesquisados.
    """
    logger.info("Selecionando classificações econômicas via termos de busca...")
    selecionados = set() # Mantido para fins de log e depuração

    try:
        # Abre o dropdown uma vez no início.
        # Se clicar numa opção fechar o dropdown, essa lógica precisaria
        # ser movida para dentro do loop 'for'.
        container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "container-classificacao"))
        )
        trigger = container.find_element(By.CSS_SELECTOR, "span.select2-selection.select2-selection--multiple")
        trigger.click()
        logger.debug("Dropdown de Classificação Econômica ativado.")

    except Exception as e:
        logger.error(f"Não foi possível abrir o container de Classificação Econômica: {e}")
        # Se não conseguir abrir o dropdown, não adianta continuar.
        return

    for termo in termos:
        logger.info(f"→ Iniciando busca por termo: '{termo}'")
        
        # Este loop 'while' agora serve principalmente para o caso de uma busca
        # retornar múltiplas opções que precisam ser clicadas uma a uma.
        # Com termos exatos, ele deve rodar apenas uma vez por termo.
        while True:
            try:
                # 1. Localiza o campo de busca e insere o termo
                input_field = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, "textarea.select2-search__field"))
                )
                input_field.clear()
                input_field.send_keys(termo)
                time.sleep(2)  # Espera para a chamada AJAX carregar as opções

                # --- BLOCO ANTIGO COMENTADO, CONFORME SOLICITADO ---
                # # 3. Pega todas as opções visíveis ainda não selecionadas
                # opcoes = [
                #     opcao for opcao in driver.find_elements(
                #         By.XPATH, "//li[contains(@class, 'select2-results__option--selectable')]"
                #     ) if opcao.get_attribute("id") not in selecionados
                # ]
                # if not opcoes:
                #     logger.info(f"✓ Nenhuma nova opção encontrada para o termo '{termo}'. Termo concluído.")
                #     break
                # opcao = opcoes[0]
                # --- FIM DO BLOCO ANTIGO ---

                # --- NOVO TRECHO PARA SEMPRE CLICAR NA PRIMEIRA OPÇÃO ---
                # Localiza e aguarda a primeira opção selecionável estar pronta para clique.
                # Se nenhuma opção aparecer em 5 segundos, uma TimeoutException será lançada.
                primeira_opcao_locator = (By.XPATH, "//li[contains(@class, 'select2-results__option--selectable')]")
                opcao = WebDriverWait(driver, 5).until(
                    EC.element_to_be_clickable(primeira_opcao_locator)
                )
                
                texto = opcao.text
                id_opcao = opcao.get_attribute("id")

                # Clica na opção encontrada
                driver.execute_script("arguments[0].scrollIntoView(true);", opcao) # Garante que está visível
                opcao.click()
                logger.info(f"✓ Opção selecionada: {texto}")
                selecionados.add(id_opcao) # Adiciona ao set para sabermos o que foi selecionado

                # Como o termo é exato, esperamos apenas um resultado.
                # O 'break' nos tira do loop 'while' e vai para o próximo termo no loop 'for'.
                break 

            except TimeoutException:
                # Este bloco agora é o caminho esperado se a busca não retornar resultados.
                logger.warning(f"Nenhuma opção encontrada para o termo '{termo}' após a busca.")
                break # Sai do loop 'while' e vai para o próximo termo.
            
            except Exception as e:
                logger.error(f"Erro inesperado ao selecionar opção com termo '{termo}': {e}")
                break # Sai do loop 'while' em caso de outro erro.

    # Clica fora da área do dropdown para garantir que ele se feche
    try:
        driver.find_element(By.TAG_NAME, "body").click()
        logger.info("Finalizada a seleção de todas as classificações.")
    except Exception:
        pass # Ignora se houver erro ao clicar no body


# ==============================================================================
# 5. DOWNLOAD CSV
# ==============================================================================

def baixar_e_salvar_csv(driver, ano: str, download_dir: str):
    try:
        botao = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@serigyitem='salvarRelatorio' and @serigyvalue='csv']"))
        )
        arquivos_antes = set(os.listdir(download_dir))
        botao.click()
        logger.info("Botão 'Exportar CSV' clicado.")

        tempo_max, tempo = 60, 0
        while tempo < tempo_max:
            novos = set(os.listdir(download_dir)) - arquivos_antes
            completos = [f for f in novos if not f.endswith(('.crdownload', '.tmp'))]
            if completos:
                nome = completos[0]
                final_dir = os.path.join("..", "data", "pacatuba", ano)
                os.makedirs(final_dir, exist_ok=True)
                caminho_origem = os.path.join(download_dir, nome)
                destino = os.path.join(final_dir, f"royalties_pacatuba_{ano}.csv")
                os.rename(caminho_origem, destino)
                logger.info(f"Arquivo salvo em: {destino}")
                return
            time.sleep(1)
            tempo += 1

        logger.warning("Tempo de espera excedido. Download não concluído.")
    except Exception as e:
        logger.error(f"Erro ao tentar baixar o CSV: {e}")

# ==============================================================================
# 6. PROCESSAMENTO DO ANO
# ==============================================================================

def processar_ano(ano: str):
    logger.info(f"=== Iniciando processamento para {ano} ===")
    download_dir = os.path.abspath(os.path.join("../data/", "pacatuba", ano))
    os.makedirs(download_dir, exist_ok=True)

    driver = start_driver(headless=False, download_dir=download_dir)
    try:
        WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))

        selecionar_dropdown(driver, "select2-tipo-container", "Pagamento")
        selecionar_dropdown(driver, "select2-ano-container", ano)

        selecionar_classificacoes_economicas(driver, TERMOS_ROYALTIES)

        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button#filtrar.btn-buscar"))
        ).click()

        WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.XPATH, "//a[@serigyitem='salvarRelatorio' and @serigyvalue='csv']"))
        )

        baixar_e_salvar_csv(driver, ano, download_dir)

    except Exception as e:
        logger.error(f"Erro ao processar ano {ano}: {e}")
    finally:
        driver.quit()
        logger.info(f"Finalizado processamento de {ano}")

# ==============================================================================
# 7. EXECUÇÃO
# ==============================================================================

ANOS = ["2021"]

for ano in ANOS:
    processar_ano(ano)


[2025-07-08 15:12:54,576] - INFO - [processar_ano] - === Iniciando processamento para 2021 ===
[2025-07-08 15:12:54,578] - INFO - [start_driver] - Iniciando driver do Chrome...
[2025-07-08 15:13:11,865] - INFO - [selecionar_dropdown] - Selecionando: Pagamento
[2025-07-08 15:13:12,102] - INFO - [selecionar_dropdown] - Selecionando: 2021
[2025-07-08 15:13:24,871] - INFO - [selecionar_classificacoes_economicas] - Selecionando classificações econômicas via termos de busca...
[2025-07-08 15:13:25,001] - INFO - [selecionar_classificacoes_economicas] - → Iniciando busca por termo: '1340040000'
[2025-07-08 15:13:27,219] - INFO - [selecionar_classificacoes_economicas] - ✓ Opção selecionada: 1340040000 - COMP. FINAN. COM ROYALTIES P/PROD. PETRO. OU G.N -TERRA
[2025-07-08 15:13:27,220] - INFO - [selecionar_classificacoes_economicas] - → Iniciando busca por termo: '1340050000'
[2025-07-08 15:13:29,396] - INFO - [selecionar_classificacoes_economicas] - ✓ Opção selecionada: 1340050000 - COMP. FINAN.